The model was automatically reconstructed from sequenced genome (retrieved with NCBI RefSeq accession code) using carveme.
The anaerobic xylose medium with yeast-extract is defined in medium.tsv, which is used to gapfill and initialise exchange reaction bounds on the model.

To generate the model:

```
carve --refseq GCF_003014985.1 --gapfill nj4_med -i nj4_med --mediadb medium.tsv -o GEMs/NJ4.xml 
```

In [14]:
from cobra.io import read_sbml_model
from utils import model_validation as mv

nj4 = read_sbml_model('GEMs/NJ4.xml')

# Testing the model:

A sanity-check on the model and its capabilities should include:

1. check whether growth is possible on the medium,  
use ```growth_possible```.

1. check for energy-gerating cycles,  
use ```energy_generation_cycle```

1. check whether the exhange reactions exist in the model,  
use ```reactions_exist```.

2. check whether the exhange reactions are blocked,  
use ```check_blocked_reactions```.

2. check whether the exhange reactions can carry flux at optimal solution (FVA),  
use ```check_production```.

In [2]:
mv.growth_possible(nj4)

True

In [42]:
mv.energy_generation_cycle(nj4)

False

In [15]:
uptake_reactions = [
    'EX_xyl__D_e', # xylose exchange
]

production_reactions = [
    'EX_btoh_e', # butanol exhange
    'EX_etoh_e', # ethanol exchange
    'EX_ac_e', # acetate exchange
    'EX_but_e', # butyrate exchange
    'EX_acetone_e', # acetone exchange
]

In [4]:
mv.reactions_exist(nj4, uptake_reactions)

True

In [5]:
mv.reactions_exist(nj4, production_reactions)

The following reactions are missing: ['EX_btoh_e', 'EX_acetone_e']


False

In [6]:
existing_production_reactions = [
    'EX_etoh_e', # ethanol exchange
    'EX_ac_e', # acetate exchange
    'EX_but_e', # butyrate exchange
]

In [7]:
mv.check_blocked_reactions(nj4, existing_production_reactions)

['EX_ac_e', 'EX_but_e']

In [8]:
mv.check_production(nj4, existing_production_reactions)

,minimum,maximum
EX_etoh_e,15.445155,15.844752
EX_ac_e,0.000000,0.000000
EX_but_e,0.000000,0.000000


The out-of-the-box carveme model:
- can grow on the medium
- does not contain the exhange reactions for butanol or acetone
- does not contain acetone or butyrate production in its solution space (blocked reactions)
- needs to produce ethanol at optimal growth

## Troubleshooting the carveme generated model 

The metabolites and reactions in the ABE fermentation pathway has been identified (see personal notes) with BiGG identifiers.
Will begin by checking which are / are not present in the model:

In [9]:
metabolites = ['btoh', '1btol', 'btal', 'btcoa', 'butpi', 'but', 'acetone', 'acac', 'aacoa', 'b2coa', '3hbcoa', 'accoa', 'pyr']

for metabolite in metabolites:
    try:
        nj4.metabolites.get_by_id(str(metabolite+"_c"))
    except KeyError:
        print(metabolite, "not in model")

acetone not in model


In [16]:
for reaction in production_reactions:
    try:
        nj4.reactions.get_by_id(reaction)
    except KeyError:
        print(reaction, "not in model")

EX_btoh_e not in model
EX_acetone_e not in model


In [19]:
metabolite = "btoh"

try:
    nj4.metabolites.get_by_id(str(metabolite+"_e"))
except KeyError:
    print(str(metabolite+"_e"), "not in model")

btoh_e not in model


- all the (intracellular) metabolites required for the correct behaviour is present in the model appart from acetone. will focus on fixing this pathway first
- extracellular butanol and butanol exchange is missing, even though intracellular butanol is present. need to add butanol transport and exhange reactions

### identifying missing reactions

use the universal model to find *candidates* for reactions that connect 2 metabolites

In [16]:
from config import ROOT_DIR

universal_model = read_sbml_model(str(ROOT_DIR / "community_modelling" / "GEMs" / "bigg_universe.xml"))

No objective coefficients in model. Unclear what should be optimized


In [11]:
def find_reaction_connections(universal_model, A, B):
    """Find the reaction(s) that connect metabolite A and B in the universal model."""
    rx_A = universal_model.metabolites.get_by_id(A).reactions
    rx_B = universal_model.metabolites.get_by_id(B).reactions

    # get list of elements in both r1 and r2
    common = list(set(rx_A).intersection(rx_B))
    return [r.id for r in common]

In [13]:
find_reaction_connections(universal_model, "acetone_e", "acetone_c")

['ACEt', 'ACETONEt2']

In [12]:
find_reaction_connections(universal_model, "acac_c", "acetone_c")

['ADCi']

In [20]:
find_reaction_connections(universal_model, "btoh_e", "btoh_c")

['BTOHt']

In [17]:
# butabol transport reaction
BTOHt = universal_model.reactions.get_by_id('BTOHt')

# acetoacetate -> acetone + CO2
ADCi = universal_model.reactions.get_by_id("ADCi")

# acetone transport reaction
ACEt = universal_model.reactions.get_by_id("ACEt")

# acetoacetyl-CoA + butyrate -> acetoacetate + butanoyl-CoA
nj4.reactions.get_by_id("BUTCT2").bounds = (-1000, 1000) # make this reaction reversible like god intended

nj4.add_reactions([BTOHt, ADCi, ACEt])
nj4.add_boundary(nj4.metabolites.get_by_id('btoh_e'), type='exchange', reaction_id='EX_btoh_e', lb=0, ub=1000);
nj4.add_boundary(nj4.metabolites.get_by_id('acetone_e'), type='exchange', reaction_id='EX_acetone_e', lb=0, ub=1000);

Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.


In [18]:
mv.check_blocked_reactions(nj4, production_reactions)

['EX_ac_e', 'EX_but_e']

the production of acetate and butyrate is still blocked - use the cobrapy gapfill function to identify possible solutions:

In [28]:
from cobra.flux_analysis import gapfill

def find_gapfilling_reactions(model, universal_model, reaction):
    """Using cobrapy gapfill function, find reactions from the universal model that when added will unblock reaction."""
    
    with model:
        model.objective = model.reactions.get_by_id(reaction)
        gapfilling_reactions = gapfill(model, universal_model)
        
    return [r.id for r in gapfilling_reactions[0]]

In [34]:
find_gapfilling_reactions(nj4, universal_model, "EX_ac_e")

['ACt5pp']

In [36]:
find_gapfilling_reactions(nj4, universal_model, "EX_but_e")

['BUTt2rpp']


add in transport reactions between extracellular and cytosolic butyrate and acetate

In [19]:
# butyrate transport reaction
BUTt = universal_model.reactions.get_by_id('BUTt')

# acetate transport reaction
ACtr = universal_model.reactions.get_by_id("ACtr")

nj4.add_reactions([BUTt, ACtr])

In [20]:
assert mv.growth_possible(nj4)
assert not mv.energy_generation_cycle(nj4)
assert not mv.check_blocked_reactions(nj4, production_reactions)

Now no reactions are blocked - all tests are passed and the first stage of model testing and troubleshooting is completed. 
Script to run curation and save model as NJ4_curated is located in utils folder.

In [21]:
from cobra.io import write_sbml_model

write_sbml_model(nj4, "GEMs/NJ4_curated.xml")

## Further testing of model

next up: look into how the model behaves

In [22]:
from cobra.io import read_sbml_model
from utils import model_validation as mv

nj4 = read_sbml_model('GEMs/NJ4_curated.xml')

production_reactions = [
    'EX_btoh_e', # butanol exhange
    'EX_etoh_e', # ethanol exchange
    'EX_ac_e', # acetate exchange
    'EX_but_e', # butyrate exchange
    'EX_acetone_e', # acetone exchange
]

In [27]:
mv.check_production(nj4, production_reactions, fraction_of_optimum=1)

,minimum,maximum
EX_btoh_e,0.0,11.153371
EX_etoh_e,0.0,22.306742
EX_ac_e,0.0,4.746594
EX_but_e,0.0,4.746594
EX_acetone_e,0.0,2.373297


- it is expected that there cannot be consumption of any of these substances since there is none available in the medium

In [28]:
with nj4:
    nj4.reactions.EX_but_e.lower_bound = -10
    nj4.reactions.EX_ac_e.lower_bound = -10
    print(mv.check_production(nj4, production_reactions, fraction_of_optimum=1))

              minimum    maximum
EX_btoh_e         0.0  16.153371
EX_etoh_e         0.0  22.306768
EX_ac_e         -10.0  14.746594
EX_but_e        -10.0  14.746594
EX_acetone_e      0.0   2.998985


- if we add butyrate and acetate to the medium, they can be used as substrate (like they are during solventogenesis) and max possible butanol and acetone increases

In [23]:
genes = {'adhE1':'WP_010890846_1', 'adhE2':'WP_010890720_1', 'ctfA': 'WP_010890847_1', 'ctfB': 'WP_010890848_1'}

for gene, protein in genes.items():
    try:
        gpr = nj4.genes.get_by_id(protein)
        print(gene, ":", [r.id for r in gpr.reactions])
    except KeyError:
        print(gene, ": not in model")

adhE1 : not in model
adhE2 : ['BTS', 'BTCOARx', 'CHOLD3', 'BTS_nadph', 'ACALD', 'ALCD4', 'ALCD19', 'ALCD2x', 'IBHH', 'ALCD19y', 'BNOCA']
ctfA : ['BUTCT', 'ACACCT', 'HXCT', 'BUTCT2']
ctfB : ['BUTCT', 'ACACCT', 'BUTCT2']


shadow prices and reduced costs:

In [46]:
substrates = ["xyl__D_e", "but_e", "ac_e"]

with nj4:
    nj4.objective = "EX_btoh_e"
    sol = nj4.optimize()
    print(sol.shadow_prices[substrates])

xyl__D_e   -1.515152
but_e      -0.363636
ac_e        0.000000
Name: shadow_prices, dtype: float64


- when butanol is objective, the negative shadow price for butyrate suggests that butyrate presence would increase butanol production

In [53]:
sol = nj4.optimize()
sol_frame = sol.to_frame()
sol_frame[sol_frame["reduced_costs"] != 0].sort_values(by="reduced_costs")

,fluxes,reduced_costs
CPPPGO,0.0,-2.356315
H2SO,0.0,-2.235043
EX_bz_e,0.0,-1.178724
EX_o2_e,0.0,-1.117521
HIBHr,0.0,-0.606363
...,...,...
CLt3_2pp,0.0,0.020401
ACt2ipp,0.0,0.020401
GTHRDt2,0.0,0.027201
MALt3r,0.0,0.040802
